In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve

In [2]:
data = np.load("/kaggle/input/commonicms/Dataset_Experiment.npy")
history = []

In [3]:
class new_model(nn.Module):
    def __init__(self, output_layer):
        super().__init__()
        self.output_layer = output_layer
        self.pretrained = models.resnet50(pretrained=True)
        self.children_list = []
        for n,c in self.pretrained.named_children():
            self.children_list.append(c)
            if n == self.output_layer:
                break

        self.net = nn.Sequential(*self.children_list)
        self.pretrained = None
        self.linear = nn.Linear(512,1)
        self.linear.cuda()
        self.sigmoid = nn.Sigmoid()
    def forward(self,x):
        channel1 = x[:,0:1024]
        channel1 = torch.reshape((channel1),(-1,1,32,32))
        channel1 = channel1/255
        channel1 = torch.cat((channel1,channel1,channel1), axis = 1)
        channel1 = self.net(channel1)
        z = channel1.mean(dim=(-2, -1))
        z = z.squeeze()
        #z = nn.Dropout(0.5)(z)
        z = self.linear(z)
        z = self.sigmoid(z)
        z = z.squeeze()
        return z
model = new_model(output_layer = 'layer2')
model = model.to('cuda:0')

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [4]:
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader
dataset_tensor = torch.from_numpy(data).float()
train_dataset, val_dataset = random_split(dataset_tensor,[448200,49800])
train_loader = DataLoader(dataset = train_dataset, batch_size=1024, shuffle=True)
val_loader = DataLoader(dataset = val_dataset, batch_size=1024, shuffle=True)

In [5]:
def find_auc():
    ypred = []
    ytrue = []
    with torch.no_grad():
            for batch in val_loader:
                x_val = batch[:,:2048]
                y_val = batch[:,2048]
                x_val = x_val.to(device)
                y_val = y_val.to(device)
                model.eval()
                yhat = model(x_val)
                yhat = yhat.detach().cpu().numpy()
                ypred.append(yhat)
                ytrue.append(y_val.cpu().numpy())
                #val_loss = loss_fn(yhat,y_val).item()
                #val_losses.append(val_loss)

    yp = np.zeros((48,1024))
    yt = np.zeros((48,1024))
    for i in range(48):
        yp[i,:] = np.array(ypred[i])
        yt[i,:] = np.array(ytrue[i])

    yp = np.reshape(yp, (-1))
    yt = np.reshape(yt,(-1))
    fpr, tpr, thresholds = roc_curve(yt, yp)
    print("Auc validation ",metrics.auc(fpr, tpr))


In [6]:
def make_train_step(model, loss_fn, optimizer):
    def train_step(x, y):
        model.train()
        yhat = model(x)
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        #return loss.item()
    return train_step
#-------------------------------------------------------
loss_fn = nn.BCELoss(reduction = "mean")
optimizer = optim.Adam(model.parameters(), lr = 0.001)
train_step = make_train_step(model, loss_fn, optimizer)
#--------------------------------------------------------
n_epochs = 20
training_losses = []
validation_losses = []
device = 'cuda'
#---------------------------------------------------------
train_loss = 0
for epoch in range(n_epochs):
    for batch in train_loader:
        x_batch = batch[:,:2048]
        y_batch = batch[:,2048]
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        train_step(x_batch, y_batch)
    #---------------------------------------------------------------------
    
    
    with torch.no_grad():
        valid_loss = 0
        for batch in val_loader:
            x_val = batch[:,:2048]
            y_val = batch[:,2048]
            x_val = x_val.to(device)
            y_val = y_val.to(device)
            model.eval()
            yhat = model(x_val)
            valid_loss += loss_fn(yhat,y_val).item()
            
        
    valid_loss /= len(val_loader)
    history.append(valid_loss)
    print(f" Validation loss: {valid_loss:.3f}")

    find_auc()
    
    state = {
        'epoch': epoch,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()}
    savepath =  f"./checkpoint{epoch}.t7"
    torch.save(state,savepath)
    print(f"{epoch+1} model saved")

 Validation loss: 0.731
Auc validation  0.7156827779996008
1 model saved
 Validation loss: 0.630
Auc validation  0.7861368187220268
2 model saved
 Validation loss: 0.568
Auc validation  0.7935399316499392
3 model saved
 Validation loss: 0.893
Auc validation  0.6221026567039276
4 model saved
 Validation loss: 0.547
Auc validation  0.7980842146385868
5 model saved
 Validation loss: 0.571
Auc validation  0.788013270225922
6 model saved
 Validation loss: 1.042
Auc validation  0.7200059139174679
7 model saved
 Validation loss: 0.896
Auc validation  0.6686869419268553
8 model saved
 Validation loss: 0.685
Auc validation  0.7444498673887651
9 model saved
 Validation loss: 0.640
Auc validation  0.7823981196210654
10 model saved
 Validation loss: 0.889
Auc validation  0.6298068760770204
11 model saved
 Validation loss: 0.650
Auc validation  0.7857916907721005
12 model saved
 Validation loss: 0.540
Auc validation  0.8043030486019156
13 model saved
 Validation loss: 1.293
Auc validation  0.524779